# Import Storage files and create readable CSVs

In [10]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, Polygon, asShape 
import plotly.graph_objects as go
import plotly.express as px
from shapely.geometry import LineString
import shapely
import matplotlib.pyplot as plt
import polyline
import csv

Define functions

In [11]:
def make_geo_dataframe_w_flex_input(df,coordinate_system):
    """ Makes geodaframe from pandas dataframe and returns it in epsg:4326 coordinates"""
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Latitude, df.Longitude))
    gdf = gdf.set_crs(coordinate_system, inplace = True)
    #gdf = gdf.to_crs('epsg:4326', inplace= True)
    return gdf

In [12]:
def get_delimiter(file_path: str) -> str:
    with open(file_path, 'r') as csvfile:
        delimiter = str(csv.Sniffer().sniff(csvfile.read()).delimiter)
        return delimiter

In [13]:
def read_df_from_csv(file_path):
    delimiter = get_delimiter(file_path)
    df = pd.read_csv(file_path, delimiter = delimiter)
    return df

In [14]:
def swap_xy(geom):
    #This function changes x and y coordinates, taken from https://gis.stackexchange.com/questions/291247/interchange-y-x-to-x-y-with-geopandas-python-or-qgis
    if geom.is_empty:
        return geom

    if geom.has_z:
        def swap_xy_coords(coords):
            for x, y, z in coords:
                yield (y, x, z)
    else:
        def swap_xy_coords(coords):
            for x, y in coords:
                yield (y, x)

    # Process coordinates from each supported geometry type
    if geom.type in ('Point', 'LineString', 'LinearRing'):
        return type(geom)(list(swap_xy_coords(geom.coords)))
    elif geom.type == 'Polygon':
        ring = geom.exterior
        shell = type(ring)(list(swap_xy_coords(ring.coords)))
        holes = list(geom.interiors)
        for pos, ring in enumerate(holes):
            holes[pos] = type(ring)(list(swap_xy_coords(ring.coords)))
        return type(geom)(shell, holes)
    elif geom.type.startswith('Multi') or geom.type == 'GeometryCollection':
        # Recursive call
        return type(geom)([swap_xy(part) for part in geom.geoms])
    else:
        raise ValueError('Type %r not recognized' % geom.type)

Import data

In [15]:
#define Filepath:
filePath = 'C:/Users/TSR/My Python projects/geospatial analysis model/Data/CO2STOP.csv'

In [16]:
#Load dataframe
df_storage = read_df_from_csv(filePath)
df_storage = df_storage[['OBJECTID','COUNTRYCODE','X','Y', 'STORE_CAP_MEAN']].copy()

#rename X Y and make geo dataframe
df_storage.rename(columns={'X':'Latitude', 'Y':'Longitude'}, inplace=True)
gdf_storage = make_geo_dataframe_w_flex_input(df_storage, 'epsg:32631')
#change coordinate system
gdf_storage.to_crs(epsg =4326, inplace=True )
gdf_storage['Longitude'] = gdf_storage.centroid.x  
gdf_storage['Latitude'] = gdf_storage.centroid.y
#swap xy coordinates: 
#gdf_storage['geometry'] = gdf_storage['geometry'].apply(lambda x: swap_xy(x))


C:\Users\TSR\AppData\Local\Temp\ipykernel_1628\3684961604.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_storage['Longitude'] = gdf_storage.centroid.x
C:\Users\TSR\AppData\Local\Temp\ipykernel_1628\3684961604.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_storage['Latitude'] = gdf_storage.centroid.y


In [17]:
gdf_storage

,OBJECTID,COUNTRYCODE,Latitude,Longitude,STORE_CAP_MEAN,geometry
0,439,GB,54.012560,1.270767,13293.914400,POINT (1.27077 54.01256)
1,440,GB,53.410405,1.662874,0.000000,POINT (1.66287 53.41041)
2,441,GB,60.261169,1.787312,370.438622,POINT (1.78731 60.26117)
3,442,GB,59.743856,1.390370,12220.080930,POINT (1.39037 59.74386)
4,443,GB,57.231001,0.931109,339.066000,POINT (0.93111 57.23100)
...,...,...,...,...,...,...
91,493,GB,58.175041,-1.689003,55.843200,POINT (-1.68900 58.17504)
92,494,GB,57.913570,-0.270809,60.830000,POINT (-0.27081 57.91357)
93,495,GB,59.576401,1.691081,390.445440,POINT (1.69108 59.57640)
94,496,GB,60.872461,1.700833,254.432640,POINT (1.70083 60.87246)


# Export data

In [18]:
save = True
if save:
# export pipeline as shapefile
        gdf_storage.to_file('C:/Users/TSR/My Python projects/geospatial analysis model/Results/Shapefiles/storage_North_EU', driver='ESRI Shapefile')
        gdf_storage.to_csv('C:/Users/TSR/My Python projects/geospatial analysis model/storage_North_EU.csv', sep=';', index=False)

c:\Users\TSR\Anaconda3\envs\testenvironment\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\TSR\AppData\Local\Temp\ipykernel_1628\301439699.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_storage.to_file('C:/Users/TSR/My Python projects/geospatial analysis model/Results/Shapefiles/storage_North_EU', driver='ESRI Shapefile')
